## Lineárna a polynomiálna regresia pre stanovenie cien tekvíc - Lekcia 3
<p >
   <img src="../../images/linear-polynomial.png"
   width="800"/>
   <figcaption>Infografika od Dasani Madipalli</figcaption>


#### Úvod

Doteraz ste preskúmali, čo je regresia, na základe vzorových údajov zo súboru údajov o cenách tekvíc, ktorý budeme používať počas celej tejto lekcie. Vizualizovali ste ju pomocou `ggplot2`.💪

Teraz ste pripravení ponoriť sa hlbšie do regresie pre strojové učenie. V tejto lekcii sa dozviete viac o dvoch typoch regresie: *základná lineárna regresia* a *polynomiálna regresia*, spolu s niektorými matematickými základmi týchto techník.

> Počas celého kurzu predpokladáme minimálne znalosti matematiky a snažíme sa ju sprístupniť študentom z iných oblastí, preto sledujte poznámky, 🧮 upozornenia, diagramy a ďalšie nástroje na učenie, ktoré vám pomôžu pochopiť.

#### Príprava

Pripomeňme si, že tieto údaje načítavate, aby ste si mohli klásť otázky.

-   Kedy je najlepší čas na kúpu tekvíc?

-   Akú cenu môžem očakávať za balenie miniatúrnych tekvíc?

-   Mal by som ich kúpiť v polovičných košoch alebo v krabici 1 1/9 bušlu? Poďme sa hlbšie ponoriť do týchto údajov.

V predchádzajúcej lekcii ste vytvorili `tibble` (moderné prepracovanie dátového rámca) a naplnili ho časťou pôvodného súboru údajov, pričom ste štandardizovali ceny podľa bušlu. Týmto spôsobom ste však dokázali zhromaždiť iba približne 400 údajových bodov a iba pre jesenné mesiace. Možno môžeme získať trochu viac detailov o povahe údajov ich dôkladnejším čistením? Uvidíme... 🕵️‍♀️

Pre túto úlohu budeme potrebovať nasledujúce balíky:

-   `tidyverse`: [tidyverse](https://www.tidyverse.org/) je [kolekcia balíkov pre R](https://www.tidyverse.org/packages), ktorá robí dátovú vedu rýchlejšou, jednoduchšou a zábavnejšou!

-   `tidymodels`: [tidymodels](https://www.tidymodels.org/) je [rámec balíkov](https://www.tidymodels.org/packages/) pre modelovanie a strojové učenie.

-   `janitor`: [janitor balík](https://github.com/sfirke/janitor) poskytuje jednoduché nástroje na skúmanie a čistenie špinavých údajov.

-   `corrplot`: [corrplot balík](https://cran.r-project.org/web/packages/corrplot/vignettes/corrplot-intro.html) poskytuje vizuálny prieskumný nástroj na korelačnú maticu, ktorý podporuje automatické preusporiadanie premenných na odhalenie skrytých vzorov medzi premennými.

Môžete ich nainštalovať pomocou:

`install.packages(c("tidyverse", "tidymodels", "janitor", "corrplot"))`

Nasledujúci skript skontroluje, či máte balíky potrebné na dokončenie tohto modulu, a v prípade ich absencie ich nainštaluje.


In [ ]:
suppressWarnings(if (!require("pacman")) install.packages("pacman"))

pacman::p_load(tidyverse, tidymodels, janitor, corrplot)

Neskôr načítame tieto skvelé balíčky a sprístupníme ich v našej aktuálnej R relácii. (Toto je len pre ilustráciu, `pacman::p_load()` to už za vás urobil)

## 1. Lineárna regresná priamka

Ako ste sa naučili v Lekcii 1, cieľom cvičenia s lineárnou regresiou je byť schopný vykresliť *priamku* *najlepšieho prispôsobenia*, aby sme:

-   **Ukázali vzťahy medzi premennými**. Zobraziť vzťah medzi premennými.

-   **Vytvárali predpovede**. Presne predpovedať, kde by nový dátový bod mohol spadnúť vo vzťahu k tejto priamke.

Na vykreslenie tohto typu priamky používame štatistickú techniku nazývanú **Regresia najmenších štvorcov**. Termín `najmenšie štvorce` znamená, že všetky dátové body okolo regresnej priamky sú umocnené na druhú a potom sčítané. Ideálne je, aby tento konečný súčet bol čo najmenší, pretože chceme mať čo najmenej chýb, teda `najmenšie štvorce`. Preto je priamka najlepšieho prispôsobenia tá priamka, ktorá nám dáva najnižšiu hodnotu súčtu štvorcov chýb - odtiaľ názov *regresia najmenších štvorcov*.

Robíme to preto, že chceme modelovať priamku, ktorá má najmenšiu kumulatívnu vzdialenosť od všetkých našich dátových bodov. Tiež umocňujeme hodnoty na druhú pred ich sčítaním, pretože nás zaujíma ich veľkosť, nie ich smer.

> **🧮 Ukáž mi matematiku**
>
> Táto priamka, nazývaná *priamka najlepšieho prispôsobenia*, môže byť vyjadrená [rovnicou](https://en.wikipedia.org/wiki/Simple_linear_regression):
>
>     Y = a + bX
>
> `X` je '`vysvetľujúca premenná` alebo `prediktor`'. `Y` je '`závislá premenná` alebo `výsledok`'. Sklon priamky je `b` a `a` je priesečník s osou y, ktorý označuje hodnotu `Y`, keď `X = 0`.
>

> ![](../../../../../../2-Regression/3-Linear/solution/images/slope.png "sklon = $y/x$")
    Infografika od Jen Looper
>
> Najprv vypočítajte sklon `b`.
>
> Inými slovami, a odkazujúc na pôvodnú otázku o údajoch o tekviciach: "predpovedajte cenu tekvice za bušel podľa mesiaca", `X` by označovalo cenu a `Y` by označovalo mesiac predaja.
>
> ![](../../../../../../2-Regression/3-Linear/solution/images/calculation.png)
    Infografika od Jen Looper
> 
> Vypočítajte hodnotu Y. Ak platíte okolo 4 dolárov, musí byť apríl!
>
> Matematika, ktorá vypočíta priamku, musí ukázať sklon priamky, ktorý tiež závisí od priesečníka, teda od toho, kde sa `Y` nachádza, keď `X = 0`.
>
> Metódu výpočtu týchto hodnôt si môžete pozrieť na webovej stránke [Math is Fun](https://www.mathsisfun.com/data/least-squares-regression.html). Navštívte tiež [tento kalkulátor najmenších štvorcov](https://www.mathsisfun.com/data/least-squares-calculator.html), aby ste videli, ako hodnoty čísel ovplyvňujú priamku.

Nie je to také strašidelné, však? 🤓

#### Korelácia

Ešte jeden pojem, ktorý treba pochopiť, je **Korelačný koeficient** medzi danými premennými X a Y. Pomocou bodového grafu môžete tento koeficient rýchlo vizualizovať. Graf s bodmi usporiadanými do úhľadnej priamky má vysokú koreláciu, ale graf s bodmi roztrúsenými všade medzi X a Y má nízku koreláciu.

Dobrý model lineárnej regresie bude taký, ktorý má vysoký (bližší k 1 ako k 0) Korelačný koeficient, použitím metódy Regresie najmenších štvorcov s regresnou priamkou.


## **2. Tanec s dátami: vytvorenie dátového rámca na modelovanie**

<p >
   <img src="../../images/janitor.jpg"
   width="700"/>
   <figcaption>Ilustrácia od @allison_horst</figcaption>


<!--![Ilustrácia od \@allison_horst](../../../../../../2-Regression/3-Linear/images/janitor.jpg){width="700"}-->


Načítajte potrebné knižnice a dataset. Konvertujte údaje na dátový rámec obsahujúci podmnožinu údajov:

-   Získajte iba tekvice ocenené na základe ceny za bušel

-   Konvertujte dátum na mesiac

-   Vypočítajte cenu ako priemer vysokých a nízkych cien

-   Konvertujte cenu tak, aby odrážala cenu za množstvo v bušeloch

> Tieto kroky sme pokryli v [predchádzajúcej lekcii](https://github.com/microsoft/ML-For-Beginners/blob/main/2-Regression/2-Data/solution/lesson_2-R.ipynb).


In [ ]:
# Load the core Tidyverse packages
library(tidyverse)
library(lubridate)

# Import the pumpkins data
pumpkins <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/2-Regression/data/US-pumpkins.csv")


# Get a glimpse and dimensions of the data
glimpse(pumpkins)


# Print the first 50 rows of the data set
pumpkins %>% 
  slice_head(n = 5)

V duchu čírej dobrodružnosti sa poďme pozrieť na [`janitor package`](../../../../../../2-Regression/3-Linear/solution/R/github.com/sfirke/janitor), ktorý poskytuje jednoduché funkcie na skúmanie a čistenie nečistých údajov. Napríklad sa pozrime na názvy stĺpcov našich údajov:


In [ ]:
# Return column names
pumpkins %>% 
  names()

🤔 Môžeme to urobiť lepšie. Poďme zmeniť názvy týchto stĺpcov na `friendR` konvertovaním na konvenciu [snake_case](https://en.wikipedia.org/wiki/Snake_case) pomocou `janitor::clean_names`. Ak sa chcete dozvedieť viac o tejto funkcii: `?clean_names`


In [ ]:
# Clean names to the snake_case convention
pumpkins <- pumpkins %>% 
  clean_names(case = "snake")

# Return column names
pumpkins %>% 
  names()

Veľa poriadku s tidyR 🧹! Teraz si zatancujeme s dátami pomocou `dplyr`, ako v predchádzajúcej lekcii! 💃


In [ ]:
# Select desired columns
pumpkins <- pumpkins %>% 
  select(variety, city_name, package, low_price, high_price, date)



# Extract the month from the dates to a new column
pumpkins <- pumpkins %>%
  mutate(date = mdy(date),
         month = month(date)) %>% 
  select(-date)



# Create a new column for average Price
pumpkins <- pumpkins %>% 
  mutate(price = (low_price + high_price)/2)


# Retain only pumpkins with the string "bushel"
new_pumpkins <- pumpkins %>% 
  filter(str_detect(string = package, pattern = "bushel"))


# Normalize the pricing so that you show the pricing per bushel, not per 1 1/9 or 1/2 bushel
new_pumpkins <- new_pumpkins %>% 
  mutate(price = case_when(
    str_detect(package, "1 1/9") ~ price/(1.1),
    str_detect(package, "1/2") ~ price*2,
    TRUE ~ price))

# Relocate column positions
new_pumpkins <- new_pumpkins %>% 
  relocate(month, .before = variety)


# Display the first 5 rows
new_pumpkins %>% 
  slice_head(n = 5)

Dobrá práca!👌 Teraz máte čistý a uprataný dátový súbor, na ktorom môžete postaviť svoj nový regresný model!

Čo tak rozptylový graf?


In [ ]:
# Set theme
theme_set(theme_light())

# Make a scatter plot of month and price
new_pumpkins %>% 
  ggplot(mapping = aes(x = month, y = price)) +
  geom_point(size = 1.6)


Bodový graf nám pripomína, že máme údaje za mesiace iba od augusta do decembra. Pravdepodobne potrebujeme viac údajov, aby sme mohli vyvodiť závery lineárnym spôsobom.

Pozrime sa znova na naše modelovacie údaje:


In [ ]:
# Display first 5 rows
new_pumpkins %>% 
  slice_head(n = 5)

Čo ak by sme chceli predpovedať `cenu` tekvice na základe stĺpcov `mesto` alebo `balenie`, ktoré sú typu znakový reťazec? Alebo ešte jednoduchšie, ako by sme mohli nájsť koreláciu (ktorá vyžaduje, aby oba vstupy boli numerické) medzi, povedzme, `balenie` a `cena`? 🤷🤷

Modely strojového učenia fungujú najlepšie s numerickými vlastnosťami namiesto textových hodnôt, takže vo všeobecnosti je potrebné previesť kategóriálne vlastnosti na numerické reprezentácie.

To znamená, že musíme nájsť spôsob, ako upraviť naše prediktory, aby ich model mohol efektívne používať, proces známy ako `inžinierstvo vlastností`.


## 3. Predspracovanie údajov na modelovanie pomocou receptov 👩‍🍳👨‍🍳

Aktivity, ktoré upravujú hodnoty prediktorov, aby ich model mohol efektívne využívať, sa nazývajú `inžinierstvo vlastností`.

Rôzne modely majú rôzne požiadavky na predspracovanie. Napríklad metóda najmenších štvorcov vyžaduje `kódovanie kategóriálnych premenných`, ako sú mesiac, odroda a city_name. To jednoducho zahŕňa `preklad` stĺpca s `kategóriálnymi hodnotami` na jeden alebo viac `numerických stĺpcov`, ktoré nahradia pôvodný.

Napríklad, predpokladajme, že vaše údaje obsahujú nasledujúcu kategóriálnu vlastnosť:

|  mesto   |
|:--------:|
| Denver   |
| Nairobi  |
|  Tokio   |

Môžete použiť *ordinálne kódovanie*, aby ste nahradili každú kategóriu jedinečnou celočíselnou hodnotou, napríklad takto:

| mesto |
|:-----:|
|   0   |
|   1   |
|   2   |

A presne to urobíme s našimi údajmi!

V tejto časti preskúmame ďalší úžasný balík Tidymodels: [recipes](https://tidymodels.github.io/recipes/) - ktorý je navrhnutý tak, aby vám pomohol predspracovať vaše údaje **predtým**, než začnete trénovať váš model. V jadre receptu je objekt, ktorý definuje, aké kroky by sa mali aplikovať na dátovú množinu, aby bola pripravená na modelovanie.

Teraz si vytvoríme recept, ktorý pripraví naše údaje na modelovanie tým, že nahradí jedinečné celé číslo za všetky pozorovania v stĺpcoch prediktorov:


In [ ]:
# Specify a recipe
pumpkins_recipe <- recipe(price ~ ., data = new_pumpkins) %>% 
  step_integer(all_predictors(), zero_based = TRUE)


# Print out the recipe
pumpkins_recipe

Úžasné! 👏 Práve sme vytvorili náš prvý recept, ktorý špecifikuje výsledok (cenu) a jeho zodpovedajúce prediktory, pričom všetky stĺpce prediktorov by mali byť zakódované ako sada celých čísel 🙌! Poďme si to rýchlo rozobrať:

-   Volanie `recipe()` s formulou určuje receptu *úlohy* premenných pomocou údajov `new_pumpkins` ako referencie. Napríklad stĺpec `price` bol priradený úlohe `outcome`, zatiaľ čo ostatné stĺpce boli priradené úlohe `predictor`.

-   `step_integer(all_predictors(), zero_based = TRUE)` špecifikuje, že všetky prediktory by mali byť konvertované na sadu celých čísel, pričom číslovanie začína od 0.

Sme si istí, že vás možno napadli myšlienky ako: "Toto je tak super!! Ale čo ak by som potreboval potvrdiť, že recepty robia presne to, čo od nich očakávam? 🤔"

To je skvelá myšlienka! Vidíte, keď je váš recept definovaný, môžete odhadnúť parametre potrebné na skutočné predspracovanie údajov a potom extrahovať spracované údaje. Zvyčajne to nemusíte robiť, keď používate Tidymodels (o chvíľu uvidíme bežnú konvenciu -> `workflows`), ale môže to byť užitočné, keď chcete vykonať určitú kontrolu správnosti, aby ste si potvrdili, že recepty robia to, čo očakávate.

Na to budete potrebovať ďalšie dva príkazy: `prep()` a `bake()`, a ako vždy, naši malí R kamaráti od [`Allison Horst`](https://github.com/allisonhorst/stats-illustrations) vám pomôžu lepšie pochopiť túto tému!

<p >
   <img src="../../images/recipes.png"
   width="550"/>
   <figcaption>Ilustrácia od @allison_horst</figcaption>


[`prep()`](https://recipes.tidymodels.org/reference/prep.html): odhaduje potrebné parametre z tréningovej množiny, ktoré môžu byť neskôr aplikované na iné dátové množiny. Napríklad, pre daný stĺpec prediktora, ktorá hodnota bude priradená ako celé číslo 0, 1, 2 atď.

[`bake()`](https://recipes.tidymodels.org/reference/bake.html): vezme pripravený recept a aplikuje operácie na akúkoľvek dátovú množinu.

Takže, poďme pripraviť a aplikovať naše recepty, aby sme naozaj potvrdili, že v pozadí budú stĺpce prediktorov najskôr zakódované pred tým, ako sa model prispôsobí.


In [ ]:
# Prep the recipe
pumpkins_prep <- prep(pumpkins_recipe)

# Bake the recipe to extract a preprocessed new_pumpkins data
baked_pumpkins <- bake(pumpkins_prep, new_data = NULL)

# Print out the baked data set
baked_pumpkins %>% 
  slice_head(n = 10)

Hurá! 🥳 Spracované dáta `baked_pumpkins` majú všetky svoje prediktory zakódované, čo potvrdzuje, že kroky predspracovania definované ako náš recept fungujú podľa očakávania. Pre vás to môže byť ťažšie na čítanie, ale pre Tidymodels je to oveľa zrozumiteľnejšie! Nájdite si chvíľu na zistenie, ktorá pozorovaná hodnota bola namapovaná na zodpovedajúce celé číslo.

Stojí tiež za zmienku, že `baked_pumpkins` je dátový rámec, na ktorom môžeme vykonávať výpočty.

Napríklad, poďme sa pokúsiť nájsť dobrú koreláciu medzi dvoma bodmi vašich dát, aby sme potenciálne vytvorili dobrý prediktívny model. Na to použijeme funkciu `cor()`. Napíšte `?cor()`, aby ste sa dozvedeli viac o tejto funkcii.


In [ ]:
# Find the correlation between the city_name and the price
cor(baked_pumpkins$city_name, baked_pumpkins$price)

# Find the correlation between the package and the price
cor(baked_pumpkins$package, baked_pumpkins$price)


Ako sa ukazuje, medzi mestom a cenou existuje iba slabá korelácia. Avšak medzi balíkom a jeho cenou je o niečo lepšia korelácia. To dáva zmysel, však? Zvyčajne platí, že čím väčšia je krabica s produktmi, tým vyššia je cena.

Keď už sme pri tom, poďme sa pokúsiť vizualizovať korelačnú maticu všetkých stĺpcov pomocou balíka `corrplot`.


In [ ]:
# Load the corrplot package
library(corrplot)

# Obtain correlation matrix
corr_mat <- cor(baked_pumpkins %>% 
                  # Drop columns that are not really informative
                  select(-c(low_price, high_price)))

# Make a correlation plot between the variables
corrplot(corr_mat, method = "shade", shade.col = NA, tl.col = "black", tl.srt = 45, addCoef.col = "black", cl.pos = "n", order = "original")

🤩🤩 Oveľa lepšie.

Dobrá otázka, ktorú si teraz môžeme položiť na základe týchto údajov, je: '`Akú cenu môžem očakávať za daný balík tekvíc?`' Poďme na to!

> Poznámka: Keď **`bake()`** použijete na pripravený recept **`pumpkins_prep`** s **`new_data = NULL`**, získate spracované (t.j. zakódované) tréningové dáta. Ak by ste mali iný súbor údajov, napríklad testovaciu množinu, a chceli by ste vidieť, ako by recept tieto údaje predspracoval, jednoducho by ste použili **`bake()`** na **`pumpkins_prep`** s **`new_data = test_set`**.

## 4. Vytvorenie lineárneho regresného modelu

<p >
   <img src="../../images/linear-polynomial.png"
   width="800"/>
   <figcaption>Infografika od Dasani Madipalli</figcaption>


Teraz, keď sme vytvorili recept a skutočne potvrdili, že údaje budú správne predspracované, poďme vytvoriť regresný model, aby sme odpovedali na otázku: `Akú cenu môžem očakávať za dané balenie tekvice?`

#### Natrénujte lineárny regresný model pomocou tréningovej množiny

Ako ste už pravdepodobne zistili, stĺpec *price* je `výsledná` premenná, zatiaľ čo stĺpec *package* je `prediktorová` premenná.

Na to najskôr rozdelíme údaje tak, že 80 % pôjde do tréningovej množiny a 20 % do testovacej množiny, potom definujeme recept, ktorý zakóduje stĺpec prediktora do množiny celých čísel, a následne vytvoríme špecifikáciu modelu. Recept nebudeme pripravovať a piecť, pretože už vieme, že údaje predspracuje podľa očakávania.


In [ ]:
set.seed(2056)
# Split the data into training and test sets
pumpkins_split <- new_pumpkins %>% 
  initial_split(prop = 0.8)


# Extract training and test data
pumpkins_train <- training(pumpkins_split)
pumpkins_test <- testing(pumpkins_split)



# Create a recipe for preprocessing the data
lm_pumpkins_recipe <- recipe(price ~ package, data = pumpkins_train) %>% 
  step_integer(all_predictors(), zero_based = TRUE)



# Create a linear model specification
lm_spec <- linear_reg() %>% 
  set_engine("lm") %>% 
  set_mode("regression")

Skvelá práca! Teraz, keď máme recept a špecifikáciu modelu, musíme nájsť spôsob, ako ich spojiť do objektu, ktorý najprv predspracuje dáta (v zákulisí prep+bake), natrénuje model na predspracovaných dátach a zároveň umožní aj prípadné aktivity po spracovaní. Čo poviete na takýto pokoj na duši!🤩

V Tidymodels sa tento praktický objekt nazýva [`workflow`](https://workflows.tidymodels.org/) a pohodlne uchováva vaše modelovacie komponenty! Toto by sme v *Python-e* nazvali *pipelines*.

Tak poďme všetko zabaliť do workflow!📦


In [ ]:
# Hold modelling components in a workflow
lm_wf <- workflow() %>% 
  add_recipe(lm_pumpkins_recipe) %>% 
  add_model(lm_spec)

# Print out the workflow
lm_wf

👌 Navyše, pracovný postup môže byť prispôsobený/vytrénovaný podobne ako model.


In [ ]:
# Train the model
lm_wf_fit <- lm_wf %>% 
  fit(data = pumpkins_train)

# Print the model coefficients learned 
lm_wf_fit

Z výstupu modelu môžeme vidieť koeficienty naučené počas tréningu. Predstavujú koeficienty priamky najlepšieho prispôsobenia, ktorá nám poskytuje najnižšiu celkovú chybu medzi skutočnou a predpovedanou premennou.

#### Vyhodnotenie výkonu modelu pomocou testovacej množiny

Je čas zistiť, ako si model viedol 📏! Ako to urobíme?

Teraz, keď sme model natrénovali, môžeme ho použiť na predpovede pre testovaciu množinu pomocou `parsnip::predict()`. Potom môžeme tieto predpovede porovnať so skutočnými hodnotami štítkov, aby sme zhodnotili, ako dobre (alebo nie!) model funguje.

Začnime vytváraním predpovedí pre testovaciu množinu a následným pripojením stĺpcov k testovacej množine.


In [ ]:
# Make predictions for the test set
predictions <- lm_wf_fit %>% 
  predict(new_data = pumpkins_test)


# Bind predictions to the test set
lm_results <- pumpkins_test %>% 
  select(c(package, price)) %>% 
  bind_cols(predictions)


# Print the first ten rows of the tibble
lm_results %>% 
  slice_head(n = 10)

Áno, práve ste natrénovali model a použili ho na predikcie!🔮 Je dobrý? Poďme vyhodnotiť výkon modelu!

V Tidymodels to robíme pomocou `yardstick::metrics()`! Pre lineárnu regresiu sa zamerajme na nasledujúce metriky:

-   `Root Mean Square Error (RMSE)`: Odmocnina z [MSE](https://en.wikipedia.org/wiki/Mean_squared_error). Poskytuje absolútnu metriku v rovnakých jednotkách ako cieľová hodnota (v tomto prípade cena tekvice). Čím menšia hodnota, tým lepší model (v jednoduchom zmysle predstavuje priemernú cenu, o ktorú sa predikcie mýlia!).

-   `Coefficient of Determination (zvyčajne známy ako R-squared alebo R2)`: Relatívna metrika, pri ktorej vyššia hodnota znamená lepšie prispôsobenie modelu. V podstate táto metrika reprezentuje, koľko variability medzi predikovanými a skutočnými hodnotami cieľovej premenné dokáže model vysvetliť.


In [ ]:
# Evaluate performance of linear regression
metrics(data = lm_results,
        truth = price,
        estimate = .pred)

Tam ide výkon modelu. Pozrime sa, či môžeme získať lepšiu predstavu vizualizáciou bodového grafu balíka a ceny, a potom použijeme predpovede na prekrytie najlepšej prispôsobenej čiary.

To znamená, že budeme musieť pripraviť a spracovať testovaciu množinu, aby sme zakódovali stĺpec balíka, a potom to spojiť s predpoveďami vytvorenými naším modelom.


In [ ]:
# Encode package column
package_encode <- lm_pumpkins_recipe %>% 
  prep() %>% 
  bake(new_data = pumpkins_test) %>% 
  select(package)


# Bind encoded package column to the results
lm_results <- lm_results %>% 
  bind_cols(package_encode %>% 
              rename(package_integer = package)) %>% 
  relocate(package_integer, .after = package)


# Print new results data frame
lm_results %>% 
  slice_head(n = 5)


# Make a scatter plot
lm_results %>% 
  ggplot(mapping = aes(x = package_integer, y = price)) +
  geom_point(size = 1.6) +
  # Overlay a line of best fit
  geom_line(aes(y = .pred), color = "orange", size = 1.2) +
  xlab("package")
  


Skvelé! Ako môžete vidieť, lineárny regresný model nedokáže veľmi dobre generalizovať vzťah medzi balíkom a jeho zodpovedajúcou cenou.

🎃 Gratulujeme, práve ste vytvorili model, ktorý dokáže predpovedať cenu niekoľkých druhov tekvíc. Vaša sviatočná tekvicová záhrada bude nádherná. Ale pravdepodobne dokážete vytvoriť ešte lepší model!

## 5. Vytvorte polynomiálny regresný model

<p >
   <img src="../../images/linear-polynomial.png"
   width="800"/>
   <figcaption>Infografika od Dasani Madipalli</figcaption>


<!--![Infografika od Dasani Madipalli](../../../../../../2-Regression/3-Linear/images/linear-polynomial.png){width="800"}-->


Niekedy naše údaje nemusia mať lineárny vzťah, no aj tak chceme predpovedať výsledok. Polynomická regresia nám môže pomôcť robiť predpovede pre zložitejšie nelineárne vzťahy.

Vezmime si napríklad vzťah medzi balením a cenou v našej dátovej sade tekvíc. Zatiaľ čo niekedy existuje lineárny vzťah medzi premennými – čím väčší objem tekvice, tým vyššia cena – niekedy tieto vzťahy nemožno znázorniť ako rovinu alebo priamku.

> ✅ Tu sú [ďalšie príklady](https://online.stat.psu.edu/stat501/lesson/9/9.8) údajov, ktoré by mohli využiť polynomickú regresiu
>
> Pozrite sa znova na vzťah medzi odrodou a cenou v predchádzajúcom grafe. Zdá sa, že by tento bodový graf mal byť nevyhnutne analyzovaný priamkou? Možno nie. V tomto prípade môžete vyskúšať polynomickú regresiu.
>
> ✅ Polynómy sú matematické výrazy, ktoré môžu pozostávať z jednej alebo viacerých premenných a koeficientov

#### Natrénujte model polynomickej regresie pomocou tréningovej množiny

Polynomická regresia vytvára *zakrivenú čiaru*, ktorá lepšie zodpovedá nelineárnym údajom.

Pozrime sa, či polynomický model bude lepší pri predpovedaní. Budeme postupovať podobným spôsobom ako predtým:

-   Vytvorte recept, ktorý špecifikuje kroky predspracovania, ktoré by sa mali vykonať na našich údajoch, aby boli pripravené na modelovanie, t. j.: kódovanie prediktorov a výpočet polynómov stupňa *n*

-   Vytvorte špecifikáciu modelu

-   Spojte recept a špecifikáciu modelu do pracovného postupu

-   Vytvorte model prispôsobením pracovného postupu

-   Vyhodnoťte, ako dobre model funguje na testovacích údajoch

Poďme na to!


In [ ]:
# Specify a recipe
poly_pumpkins_recipe <-
  recipe(price ~ package, data = pumpkins_train) %>%
  step_integer(all_predictors(), zero_based = TRUE) %>% 
  step_poly(all_predictors(), degree = 4)


# Create a model specification
poly_spec <- linear_reg() %>% 
  set_engine("lm") %>% 
  set_mode("regression")


# Bundle recipe and model spec into a workflow
poly_wf <- workflow() %>% 
  add_recipe(poly_pumpkins_recipe) %>% 
  add_model(poly_spec)


# Create a model
poly_wf_fit <- poly_wf %>% 
  fit(data = pumpkins_train)


# Print learned model coefficients
poly_wf_fit

  

#### Vyhodnotenie výkonu modelu

👏👏Vytvorili ste polynomiálny model, poďme urobiť predpovede na testovacej množine!


In [ ]:
# Make price predictions on test data
poly_results <- poly_wf_fit %>% predict(new_data = pumpkins_test) %>% 
  bind_cols(pumpkins_test %>% select(c(package, price))) %>% 
  relocate(.pred, .after = last_col())


# Print the results
poly_results %>% 
  slice_head(n = 10)

Hurá, poďme vyhodnotiť, ako model fungoval na testovacej množine pomocou `yardstick::metrics()`.


In [ ]:
metrics(data = poly_results, truth = price, estimate = .pred)

🤩🤩 Oveľa lepší výkon.

`rmse` kleslo z približne 7 na približne 3, čo naznačuje zníženie chyby medzi skutočnou cenou a predpovedanou cenou. Môžete to *voľne* interpretovať tak, že nesprávne predpovede sú v priemere nesprávne o približne 3 $. `rsq` sa zvýšilo z približne 0,4 na 0,8.

Všetky tieto metriky naznačujú, že polynomiálny model funguje oveľa lepšie ako lineárny model. Skvelá práca!

Pozrime sa, či to dokážeme vizualizovať!


In [ ]:
# Bind encoded package column to the results
poly_results <- poly_results %>% 
  bind_cols(package_encode %>% 
              rename(package_integer = package)) %>% 
  relocate(package_integer, .after = package)


# Print new results data frame
poly_results %>% 
  slice_head(n = 5)


# Make a scatter plot
poly_results %>% 
  ggplot(mapping = aes(x = package_integer, y = price)) +
  geom_point(size = 1.6) +
  # Overlay a line of best fit
  geom_line(aes(y = .pred), color = "midnightblue", size = 1.2) +
  xlab("package")


Môžete vidieť zakrivenú čiaru, ktorá lepšie zodpovedá vašim údajom! 🤩

Môžete ju urobiť ešte plynulejšou tým, že do `geom_smooth` zadáte polynomiálny vzorec, napríklad takto:


In [ ]:
# Make a scatter plot
poly_results %>% 
  ggplot(mapping = aes(x = package_integer, y = price)) +
  geom_point(size = 1.6) +
  # Overlay a line of best fit
  geom_smooth(method = lm, formula = y ~ poly(x, degree = 4), color = "midnightblue", size = 1.2, se = FALSE) +
  xlab("package")

Podobne ako hladká krivka!🤩

Tu je postup, ako vytvoriť novú predikciu:


In [ ]:
# Make a hypothetical data frame
hypo_tibble <- tibble(package = "bushel baskets")

# Make predictions using linear model
lm_pred <- lm_wf_fit %>% predict(new_data = hypo_tibble)

# Make predictions using polynomial model
poly_pred <- poly_wf_fit %>% predict(new_data = hypo_tibble)

# Return predictions in a list
list("linear model prediction" = lm_pred, 
     "polynomial model prediction" = poly_pred)


Predikcia pomocou `polynomial model` dáva zmysel, vzhľadom na bodové grafy `price` a `package`! A ak je tento model lepší ako ten predchádzajúci, pri pohľade na tie isté údaje, budete musieť plánovať rozpočet na tieto drahšie tekvice!

🏆 Skvelá práca! Vytvorili ste dva regresné modely v jednej lekcii. V poslednej časti o regresii sa naučíte o logistickej regresii na určenie kategórií.

## **🚀Výzva**

Otestujte niekoľko rôznych premenných v tomto notebooku, aby ste zistili, ako korelácia súvisí s presnosťou modelu.

## [**Kvíz po prednáške**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/14/)

## **Prehľad & Samoštúdium**

V tejto lekcii sme sa naučili o lineárnej regresii. Existujú aj iné dôležité typy regresie. Prečítajte si o technikách Stepwise, Ridge, Lasso a Elasticnet. Dobrou možnosťou na štúdium je [Stanford Statistical Learning course](https://online.stanford.edu/courses/sohs-ystatslearning-statistical-learning).

Ak sa chcete dozvedieť viac o používaní úžasného frameworku Tidymodels, pozrite si nasledujúce zdroje:

-   Webová stránka Tidymodels: [Začnite s Tidymodels](https://www.tidymodels.org/start/)

-   Max Kuhn a Julia Silge, [*Tidy Modeling with R*](https://www.tmwr.org/)*.*

###### **ĎAKUJEME:**

[Allison Horst](https://twitter.com/allison_horst?lang=en) za vytvorenie úžasných ilustrácií, ktoré robia R prístupnejším a zábavnejším. Viac ilustrácií nájdete v jej [galérii](https://www.google.com/url?q=https://github.com/allisonhorst/stats-illustrations&sa=D&source=editors&ust=1626380772530000&usg=AOvVaw3zcfyCizFQZpkSLzxiiQEM).



---

**Upozornenie**:  
Tento dokument bol preložený pomocou služby na automatický preklad [Co-op Translator](https://github.com/Azure/co-op-translator). Aj keď sa snažíme o presnosť, upozorňujeme, že automatické preklady môžu obsahovať chyby alebo nepresnosti. Pôvodný dokument v jeho pôvodnom jazyku by mal byť považovaný za autoritatívny zdroj. Pre dôležité informácie sa odporúča profesionálny ľudský preklad. Nezodpovedáme za akékoľvek nedorozumenia alebo nesprávne interpretácie vyplývajúce z použitia tohto prekladu.
